# Read and clean a single file
---------
Nombre: **Julio Enrique Sanabria Quintana**

In [1]:
import os
import sys
import logging
import pandas as pd
import unidecode


from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'esp_bigdata'
directory = 'BigData'

In [2]:
def unificando_columnas(data):
    data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'FECHA_DESPACHO_518': 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'TIPO DE INCIDIDENT'                : 'TIPO_INCIDENTE',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },inplace=True)
    return(data)

## Read Metadata
-----
General info for each relevant column

In [3]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [4]:
get_input_data?

Signature:
get_input_data(
    bucket='esp_bigdata',
    initial_directory='BigData',
    step='raw',
    filename='datos-abiertos-agosto-2019.csv',
)
Docstring:
Read a csv file in a bucket of GCS, the file must use latin1 encoding and the separator is a semicolon (;)

Args:
    bucket (str, optional): Name of the bucket. Defaults to 'esp_bigdata'.
    initial_directory (str, optional): project directory. Defaults to 'BigData'.
    filename (str, optional):csv file to read. Defaults to 'datos-abiertos-agosto-2019.csv'.

Returns:
    pandas.dataframe: dataframe with the raw data
File:      ~/ESEIT_BigData/notebooks/data_manipulator.py
Type:      function


In [5]:
julio_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-julio-2019.csv')
print(julio_2019.info())
julio_2019 = unificando_columnas(julio_2019)
#raw_data.head()
#raw_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INCIDENTE                    14560 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  14560 non-null  object
 2   CODIGO DE LOCALIDAD                14560 non-null  int64 
 3   LOCALIDAD                          14560 non-null  object
 4   EDAD                               14560 non-null  object
 5   UNIDAD                             14560 non-null  object
 6   GENERO                             14560 non-null  object
 7   RED                                14560 non-null  object
 8   TIPO_INCIDENTE                     14560 non-null  object
 9   PRIORIDAD                          14560 non-null  object
 10  MES                                14560 non-null  object
 11  CLASIFICACION FINAL                13035 non-null  object
dtypes: i

It seems that this file contains already the fields, but need to rename themhead

### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [6]:
julio_2019['LOCALIDAD'].value_counts(dropna=False) 
#dropna false para que me muetre los errores, "True" (defecto) no muetra errores

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [7]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces ("strip"quita espacios en blanco)
julio_2019['LOCALIDAD'].apply(     #apply para aplicar la función a toda la tabla
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [8]:
julio_2019['LOCALIDAD'] = julio_2019['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [9]:
julio_2019.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [10]:
col = 'GENERO'
julio_2019[col] = julio_2019[col].apply(lambda x: x.strip())
julio_2019[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [11]:
col = 'TIPO_INCIDENTE'
julio_2019[col] = julio_2019[col].apply(lambda x: x.strip())
julio_2019[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    4261
Inconsciente/Paro Cardiorrespiratorio        1623
Dificultad Respiratoria                      1398
Enfermo                                      1131
Convulsiones                                 1011
Heridos                                       747
Trastorno mental                              709
Dolor torácico                                611
Lesiones personales                           547
Intento de suicidio                           534
Patología Gineco - obstétrica                 425
Síntomas gastrointestinales                   344
Accidente cerebro vascular                    323
Caída                                         318
Intoxicaciones                                141
Ideas de suicidio                             111
Violencia Sexual                               41
Muerte Natural                                 39
SIN_DATO                                       39
Incendio estructural                           29


In [12]:
col = 'CLASIFICACION_FINAL'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
julio_2019[col].value_counts(dropna=False)

Traslado                       6162
Cancelado                      3637
NaN                            1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [13]:
julio_2019[col].fillna('SIN_DATO').value_counts()

Traslado                       6162
Cancelado                      3637
SIN_DATO                       1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [14]:
col = 'CLASIFICACION_FINAL'
julio_2019[col].fillna('SIN_DATO', inplace=True)
julio_2019[col] = julio_2019[col].apply(lambda x: x.strip())
julio_2019.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [15]:
profile = ProfileReport(df=julio_2019)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [16]:
print('Number of rows before cleaning:', julio_2019.shape[0] )
julio_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', julio_2019.shape[0] )

Number of rows before cleaning: 14560
Number of rows after cleaning: 13842


### Save the final table
use to_csv to save the table

In [86]:
# saving in a local directory
julio_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-julio-2019.csv', encoding='latin1', sep=';', index=False)

### ***datos-abiertos-agosto-2019.csv***

In [18]:
agosto_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-agosto-2019.csv')
agosto_2019 = unificando_columnas(agosto_2019)
print(agosto_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  18141 non-null  object
 2   CODIGO_LOCALIDAD                   18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO_INCIDENTE                     18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION_FINAL                18138 non-null  object
dtypes: i

In [19]:
agosto_2019['LOCALIDAD'].apply(
    lambda x: x.strip()
).value_counts()

Kennedy               2717
Engativa              1805
Suba                  1678
Bosa                  1264
Ciudad Bolivar        1220
Puente Aranda          977
San Cristobal          906
Usaquen                894
Rafael Uribe Uribe     892
Fontibon               878
Chapinero              707
Santa Fe               682
Usme                   637
Barrios Unidos         608
Teusaquillo            599
Tunjuelito             595
Los Martires           593
Antonio Nariño         369
La Candelaria          119
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [20]:
agosto_2019['LOCALIDAD'] = agosto_2019['LOCALIDAD'].apply(lambda x: x.strip())

In [21]:
agosto_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(agosto_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

In [22]:
col = 'GENERO'
agosto_2019[col] = agosto_2019[col].apply(lambda x: x.strip())

In [23]:
col = 'TIPO_INCIDENTE'
agosto_2019[col] = agosto_2019[col].apply(lambda x: x.strip())

In [24]:
col = 'CLASIFICACION_FINAL'
agosto_2019[col].fillna('SIN_DATO', inplace=True)
agosto_2019[col] = agosto_2019[col].apply(lambda x: x.strip())

In [25]:
agosto_2019[col].fillna('SIN_DATO').value_counts()

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
SIN_DATO                          3
Name: CLASIFICACION_FINAL, dtype: int64

In [26]:
print('Number of rows before cleaning:', agosto_2019.shape[0] )
agosto_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', agosto_2019.shape[0] )

Number of rows before cleaning: 18141
Number of rows after cleaning: 16807


In [27]:
# saving in a local directory
agosto_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-agosto-2019.csv', encoding='latin1', sep=';', index=False)

### **datos-abiertos-noviembre-2019.csv**

In [28]:
noviembre_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-noviembre_2019.csv')
noviembre_2019 = unificando_columnas(noviembre_2019)
print(noviembre_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15926 entries, 0 to 15925
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   15926 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  15926 non-null  object
 2   CODIGO_LOCALIDAD                   15926 non-null  int64 
 3   LOCALIDAD                          15926 non-null  object
 4   EDAD                               15926 non-null  int64 
 5   UNIDAD                             15926 non-null  object
 6   GENERO                             15926 non-null  object
 7   RED                                15926 non-null  object
 8   TIPO_INCIDENTE                     15926 non-null  object
 9   PRIORIDAD                          15926 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


In [29]:
noviembre_2019['LOCALIDAD'].apply(
    lambda x: x.strip()
).value_counts()

Kennedy               2192
Engativa              1571
Suba                  1424
Bosa                  1189
Ciudad Bolivar        1123
Puente Aranda          854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
Santa Fe               613
Chapinero              577
Usme                   577
Los Martires           560
Tunjuelito             551
Teusaquillo            546
Barrios Unidos         520
Antonio Nariño         338
La Candelaria          102
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [30]:
noviembre_2019['LOCALIDAD'] = noviembre_2019['LOCALIDAD'].apply(lambda x: x.strip())

In [31]:
noviembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(noviembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

In [32]:
col = 'GENERO'
noviembre_2019[col] = noviembre_2019[col].apply(lambda x: x.strip())

In [33]:
col = 'TIPO_INCIDENTE'
noviembre_2019[col] = noviembre_2019[col].apply(lambda x: x.strip())

In [34]:
noviembre_2019[col].fillna('SIN_DATO').value_counts()

Accidente de tránsito con heridos/Muertos    5577
Inconsciente/Paro Cardiorrespiratorio        1587
Dificultad Respiratoria                      1400
Enfermo                                      1066
Convulsiones                                  979
Heridos                                       729
Lesiones personales                           675
Trastorno mental                              601
Intento de suicidio                           600
Dolor torácico                                577
Patología Gineco - obstétrica                 373
Síntomas gastrointestinales                   355
Caída                                         346
Accidente cerebro vascular                    318
Intoxicaciones                                137
SIN_DATO                                      122
Ideas de suicidio                              90
Solicitud Apoyo / Desacato                     83
Incendio estructural                           48
Muerte Natural                                 41


In [35]:
print('Number of rows before cleaning:', noviembre_2019.shape[0] )
noviembre_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', noviembre_2019.shape[0] )

Number of rows before cleaning: 15926
Number of rows after cleaning: 15886


In [36]:
# saving in a local directory
noviembre_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-noviembre-2019.csv', encoding='latin1', sep=';', index=False)

### **datos-abiertos-octubre-2019.csv**

In [37]:
octubre_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-octubre-2019.csv')
octubre_2019 = unificando_columnas(octubre_2019)
print(octubre_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18195 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  18195 non-null  object
 2   CODIGO_LOCALIDAD                   18195 non-null  int64 
 3   LOCALIDAD                          18195 non-null  object
 4   EDAD                               18195 non-null  object
 5   UNIDAD                             9503 non-null   object
 6   GENERO                             9543 non-null   object
 7   RED                                18195 non-null  object
 8   TIPO_INCIDENTE                     18195 non-null  object
 9   PRIORIDAD                          18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


In [38]:
octubre_2019['LOCALIDAD'].apply(
    lambda x: x.strip()
).value_counts()

Kennedy               2546
Engativa              1791
Suba                  1742
Bosa                  1334
Ciudad Bolivar        1141
Puente Aranda          989
Rafael Uribe Uribe     949
Usaquen                888
Fontibon               863
San Cristobal          844
Chapinero              763
Los Martires           697
Santa Fe               689
Barrios Unidos         680
Usme                   609
Tunjuelito             589
Teusaquillo            536
Antonio Nariño         419
La Candelaria          120
Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [39]:
octubre_2019['LOCALIDAD'] = octubre_2019['LOCALIDAD'].apply(lambda x: x.strip())

In [40]:
octubre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(octubre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

In [41]:
col = 'TIPO_INCIDENTE'
octubre_2019[col] = octubre_2019[col].apply(lambda x: x.strip())

In [42]:
octubre_2019[col].fillna('SIN_DATO').value_counts()

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


In [43]:
col = 'PRIORIDAD'
octubre_2019[col] = octubre_2019[col].apply(lambda x: x.strip())

In [44]:
col = 'RED'
octubre_2019[col].fillna('SIN_DATO', inplace=True)
octubre_2019[col] = octubre_2019[col].apply(lambda x: x.strip())

In [45]:
print('Number of rows before cleaning:', octubre_2019.shape[0] )
octubre_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', octubre_2019.shape[0] )

Number of rows before cleaning: 18195
Number of rows after cleaning: 16559


In [46]:
# saving in a local directory
octubre_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-octubre-2019.csv', encoding='latin1', sep=';', index=False)

### ***datos-abiertos-septiembre-2019.csv***

In [47]:
septiembre_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-septiembre-2019.csv')
septiembre_2019 = unificando_columnas(septiembre_2019)
print(septiembre_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17018 entries, 0 to 17017
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   17018 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  17018 non-null  object
 2   CODIGO_LOCALIDAD                   17018 non-null  int64 
 3   LOCALIDAD                          17018 non-null  object
 4   EDAD                               17018 non-null  int64 
 5   UNIDAD                             9371 non-null   object
 6   GENERO                             17018 non-null  object
 7   RED                                17018 non-null  object
 8   TIPO_INCIDENTE                     17018 non-null  object
 9   PRIORIDAD                          17018 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.3+ MB
None


In [48]:
septiembre_2019['LOCALIDAD'].apply(
    lambda x: x.strip()
).value_counts()

Kennedy               2398
Suba                  1634
Engativa              1631
Bosa                  1294
Ciudad Bolivar        1138
Puente Aranda          938
Rafael Uribe Uribe     854
Usaquen                829
San Cristobal          826
Fontibon               809
Chapinero              749
Barrios Unidos         616
Tunjuelito             603
Los Martires           593
Usme                   572
Santa Fe               549
Teusaquillo            540
Antonio Nariño         362
La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [49]:
septiembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(septiembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

In [50]:
col = 'GENERO'
septiembre_2019[col] = septiembre_2019[col].apply(lambda x: x.strip())

In [51]:
col = 'TIPO_INCIDENTE'
septiembre_2019[col] = septiembre_2019[col].apply(lambda x: x.strip())

In [52]:
septiembre_2019[col].fillna('SIN_DATO').value_counts()

ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS    6333
INCONSCIENTE/PARO CARDIORRESPIRATORIO        1720
DIFICULTAD RESPIRATORIA                      1386
ENFERMO                                      1137
CONVULSIONES                                 1039
HERIDOS                                       834
INTENTO DE SUICIDIO                           653
DOLOR TORÁCICO                                629
TRASTORNO MENTAL                              621
LESIONES PERSONALES                           593
PATOLOGÍA GINECO - OBSTÉTRICA                 421
SÍNTOMAS GASTROINTESTINALES                   360
ACCIDENTE CEREBRO VASCULAR                    352
CAÍDA                                         334
INTOXICACIONES                                156
IDEAS DE SUICIDIO                             113
VIOLENCIA SEXUAL                               48
INCENDIO ESTRUCTURAL                           40
SIN_DATO                                       32
SANGRADO VAGINAL                               25


In [53]:
col = 'UNIDAD'
septiembre_2019[col].fillna('SIN_DATO', inplace=True)
septiembre_2019[col] = septiembre_2019[col].apply(lambda x: x.strip())

In [54]:
print('Number of rows before cleaning:', septiembre_2019.shape[0] )
septiembre_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', septiembre_2019.shape[0] )

Number of rows before cleaning: 17018
Number of rows after cleaning: 16267


In [55]:
septiembre_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-septiembre-2019.csv', encoding='latin1', sep=';', index=False)

### **datos-abiertos-septiembre-2019.csv**

In [57]:
diciembre_2019 = get_input_data(
    bucket= bucket,
    initial_directory=directory,
    filename = 'datos_abiertos_cons_diciembre_2019.csv')
diciembre_2019 = unificando_columnas(diciembre_2019)
print(diciembre_2019.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16243 entries, 0 to 16242
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   16243 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  16243 non-null  object
 2   CODIGO_LOCALIDAD                   16243 non-null  int64 
 3   LOCALIDAD                          16243 non-null  object
 4   EDAD                               16243 non-null  int64 
 5   UNIDAD                             16243 non-null  object
 6   GENERO                             16243 non-null  object
 7   RED                                16243 non-null  object
 8   TIPO_INCIDENTE                     16243 non-null  object
 9   PRIORIDAD                          16243 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


In [58]:
diciembre_2019['LOCALIDAD'].apply(
    lambda x: x.strip()
).value_counts()

Kennedy               2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [59]:
diciembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"] = pd.to_datetime(diciembre_2019["FECHA_INICIO_DESPLAZAMIENTO_MOVIL"],dayfirst=True)

In [63]:
diciembre_2019['LOCALIDAD'] = diciembre_2019['LOCALIDAD'].apply(lambda x: x.strip())

In [64]:
col = 'GENERO'
diciembre_2019[col] = diciembre_2019[col].apply(lambda x: x.strip())

In [65]:
col = 'TIPO_INCIDENTE'
diciembre_2019[col] = diciembre_2019[col].apply(lambda x: x.strip())

In [66]:
diciembre_2019[col].fillna('SIN_DATO').value_counts()

Heridos accidentales                      7390
Inconsciente / paro cardiorespiratorio    1743
Evento respiratorio                       1404
Convulsión                                1027
Enfermo                                    873
Trastorno mental                           650
Dolor torácico                             611
Intento de Suicidio                        543
Caída de Altura                            377
Patología ginecobstétrica                  371
Accidente Cerebro\nVascular                365
Síntomas gastrointestinales                275
Intoxicación                               153
S/D                                        138
Accidente Tránsito con Heridos/Muertos     108
Amenaza de\nSuicidio                        95
Violencia Sexual                            28
Sangrado Vaginal                            24
Lesiones Personales                         23
Quemaduras                                  16
Electrocución / Rescate                     10
Accidente Cer

In [67]:
col = 'RED'
diciembre_2019[col].fillna('SIN_DATO', inplace=True)
diciembre_2019[col] = diciembre_2019[col].apply(lambda x: x.strip())

In [68]:
print('Number of rows before cleaning:', diciembre_2019.shape[0] )
diciembre_2019.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', diciembre_2019.shape[0] )

Number of rows before cleaning: 16243
Number of rows after cleaning: 16162


In [69]:
diciembre_2019.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos-diciembre-2019.csv', encoding='latin1', sep=';', index=False)

**reporte por meses**

In [71]:
r_diciembre = ProfileReport(df=diciembre_2019)
r_diciembre.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_diciembre_2019.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [72]:
r_agosto = ProfileReport(df=agosto_2019)
r_agosto.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_agosto_2019.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
r_noviembre = ProfileReport(df=noviembre_2019)
r_noviembre.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_noviembre_2019.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
r_octubre = ProfileReport(df=octubre_2019)
r_octubre.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_octubre_2019.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [75]:
r_septiembre = ProfileReport(df=septiembre_2019)
r_septiembre.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_septiembre_2019.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [80]:
informe = pd.concat([julio_2019,agosto_2019,septiembre_2019,octubre_2019,noviembre_2019,diciembre_2019])
informe.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL,NUMERO_INCIDENTE
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado,NaN
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado,NaN
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado,NaN
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado,NaN


In [81]:
eliminarAcentos = lambda str: str.translate(str.maketrans("áàäéèëíìïòóöùúüÀÁÄÈÉËÌÍÏÒÓÖÙÚÜ", "aaaeeeiiiooouuuAAAEEEIIIOOOUUU"))

In [82]:
print('Number of rows before cleaning:', informe.shape[0] )
informe.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', informe.shape[0] )

Number of rows before cleaning: 95523
Number of rows after cleaning: 95523


In [84]:
informe.to_csv(project_dir + '/data/processed/' +'clean_datos-abiertos.csv', encoding='latin1', sep=';', index=False)